In [3]:
import os
import pandas as pd
import numpy as np

In [4]:
def decode_baby_col(row):    
    row = row.strip()
    parent = row.split(')')
    start = int(parent[0].split('(')[1].strip())
    remaining = ')'.join(parent[1:])
    remaining = remaining.strip(' ')
    find_format = remaining.split('%')
    dtypes_and_name = find_format[0].strip(' ')
    types = [t for t in dtypes_and_name.split(' ') if t]
    assert len(types) == 2, types
    dtype = types[0].strip(' ')
    name = types[1].strip(' ')
    num_c = int(float(find_format[1].split(' ')[0].strip(' ')[:-1]))
    end_of_str = 'f'.join(find_format[1].split('f')[1:])
    end_of_str = end_of_str.strip(' \r\n')        
    return {
        'start': start,
        'end': start + num_c,
        'dtype': dtype,
        'name': name,
        'desc': end_of_str.strip('"')
    }

def read_dct_file(dct):
    """
    read dct file which defines field encoded in main text files
    """
    with open(dct) as r:
        chars_to_tab = r.readlines()
        col_data = [decode_baby_col(c) for c in chars_to_tab if c.startswith('_column')]
        
    col_data_df = pd.DataFrame(col_data)
    start = 1
    columns = []
    column_widths = []
    for i, c in col_data_df.iterrows():
        s = c['start']
        e = c['end']
        if s != start:
            column_widths.append(s - start)
            columns.append('blank{0}'.format(i))
        columns.append(c['name'])
        column_widths.append(e - s)
        start = e
    return col_data_df, columns, column_widths

In [5]:
def read_row_to_table(rowstr, column_df):
    row = {}
    for idx, data in column_df.iterrows():
        info = rowstr[data['start'] - 1: data['end'] - 1]
        # print(info, data['name'])
        if data['dtype'] == 'int' or data['dtype'] == 'byte':
            if not info.strip():
                info = np.nan
            else:
                info = int(info)
        elif data['dtype'] == 'double':
            info = float(info)
        elif data['dtype'] == 'str1':
            info = str(info)
        else:
            info = str(info)
        row[data['name']] = info
    return row

# def read_nat_data(file):
#     with open(file) as r:
#         for idx, l in enumerate(r):            
#             data = read_row_to_table(l, column_reading)
#             data['idx'] = idx            
#             yield data


# a = read_nat_data('Nat2018PublicUS.c20190509.r20190717.txt')

In [ ]:
# a2 = pd.read_fwf(
#     'Nat2018PublicUS.c20190509.r20190717.txt', 
#     sep='', header=None, chunksize=10000
# )

In [ ]:
# file_as_df = []
# for idx, d in enumerate(a):
#     file_as_df.append(d)
#     if idx % 100000 == 0:
#         print(idx)

In [ ]:
# column_reading = column_reading.sort_values(by=['start'])


In [13]:
def check_int(x):
    if not isinstance(x, int) and not isinstance(x, float):
        x = int(x) if not x.strip() else np.nan
    return x

def check_double(x):
    if isinstance(x, int):
        x = float(x)
    elif not isinstance(x, float):
        x = float(x) if not x.strip() else np.nan
    return x

def read_data(file, columns, col_widths, col_df, chunk=1000000):
    desired_col = col_df['name'].values
    a2 = pd.read_fwf(
        file, sep='', 
        header=None, chunksize=chunk, widths=col_widths
    )
    groups = col_df.groupby(by=['dtype'])
    
    for df in a2:
        df.columns = columns
        
        for d, grp in groups:
            if d == 'int' or d == 'byte':
                df[grp['name'].values] = df[grp['name'].values].applymap(check_int)
            elif d == 'double':
                df[grp['name'].values] = df[grp['name'].values].applymap(check_double)
        
        yield df[desired_col]
        

In [7]:
dirs = [f for f in os.listdir('/mnt/nas/natality/') if os.path.isdir('/mnt/nas/natality/' + f)]

In [8]:
def get_schema_from_year(year):
    if not os.path.isdir('/mnt/nas/natality/' + str(year)):
        return None
    for f in os.listdir('/mnt/nas/natality/' + str(year)):
        if f.endswith('.dct'):
            column_df, columns, column_widths = read_dct_file(
                os.path.join('/mnt/nas/natality/', str(year), f)
            )
            return column_df, columns, column_widths

In [9]:
schemas = {}
for i in range(1990, 2020):
    c = get_schema_from_year(i)
    if c:
        c = c[0].set_index('name')['desc']
        schemas[i] = c

In [10]:
schema_columns_shared_across_years = pd.concat(schemas, axis=1, sort=True)

In [77]:
total_col = schema_columns_shared_across_years.shape[1]
cols_shared = ((~schema_columns_shared_across_years.isnull()).sum(axis=1)/total_col).sort_values()

In [103]:
decade_mid = schema_columns_shared_across_years[list(range(2014, 2019))].dropna(how='all', axis=0)

In [ ]:
# from plotly.offline import iplot, init_notebook_mode
# from plotly import graph_objs as go
# init_notebook_mode()

# iplot([
#     go.Heatmap(
#         z=(~decade_mid.isnull()).applymap(lambda x: int(x)).values,
#         x=decade_mid.columns,
#         y=decade_mid.index
#     )
# ])

In [11]:
schema_columns_shared_across_years.to_csv('/mnt/nas/natality/natality_schema_table.txt', sep='\t')

In [16]:
def process_data_from_year(year, chunks=10000):
    column_df, columns, column_widths = get_schema_from_year(year)
    files_in_folder = os.listdir(os.path.join('/mnt/nas/natality', str(year)))
    file_with_nat_data = [f for f in files_in_folder if f.endswith('.txt') and f.startswith('Nat')]
    assert len(file_with_nat_data) == 1, 'Failure finding Nat data for ' + str(year)
    a = read_data(
        os.path.join('/mnt/nas/natality', str(year), file_with_nat_data[0]),
        columns, column_widths, column_df, chunk=chunks
    )
    
    dfs = []
    total=0
    if not os.path.isdir('/mnt/nas/natality/processed'):
        os.mkdir('/mnt/nas/natality/processed')
    out_path = os.path.join('/mnt/nas/natality/processed', 'Nat_{0}_processed.txt'.format(year))
    for i, g in enumerate(a):    
        # dfs.append(g)        
        total += g.shape[0]
        print(i, 'processed', total)
        if i == 0:
            g.to_csv(out_path, header=True, sep='\t', mode='w')
        else:
            g.to_csv(out_path, header=False, sep='\t', mode='a')
    # return pd.concat(dfs)


In [ ]:
process_data_from_year(2015)

0 processed 10000
1 processed 20000
2 processed 30000
3 processed 40000
4 processed 50000
5 processed 60000
6 processed 70000
7 processed 80000
8 processed 90000
9 processed 100000
10 processed 110000
11 processed 120000
12 processed 130000
13 processed 140000
14 processed 150000
15 processed 160000
16 processed 170000
17 processed 180000
18 processed 190000
19 processed 200000
20 processed 210000
21 processed 220000
22 processed 230000
23 processed 240000
24 processed 250000
25 processed 260000
26 processed 270000
27 processed 280000
28 processed 290000
29 processed 300000
30 processed 310000
31 processed 320000
32 processed 330000
33 processed 340000
34 processed 350000
35 processed 360000
36 processed 370000
37 processed 380000
38 processed 390000
39 processed 400000
40 processed 410000
41 processed 420000
42 processed 430000
43 processed 440000
44 processed 450000
45 processed 460000
46 processed 470000
47 processed 480000
48 processed 490000
49 processed 500000
50 processed 510000

In [ ]:
data_2016

In [ ]:
del data_2016

In [ ]:
data_2015 = process_data_from_year(2015)